# VacationPy
----

#### Note
* Keep an eye on your API usage. Use https://developers.google.com/maps/reporting/gmp-reporting as reference for how to monitor your usage and billing.

* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [49]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [50]:
city_df = pd.read_csv("Data/cities.csv")
city_df = city_df.drop('City_ID', axis=1)
city_df.dtypes

City           object
Cloudiness      int64
Country        object
Date            int64
Humidity        int64
Lat           float64
Lng           float64
Max Temp      float64
Wind Speed    float64
dtype: object

### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [51]:
fig = gmaps.figure()
humidity = city_df['Humidity']
coord = city_df[['Lat', 'Lng']]
humidity_intensity = humidity.max()
heat_layer = gmaps.heatmap_layer(coord, weights=humidity, point_radius=10, max_intensity=humidity_intensity)
fig.add_layer(heat_layer)
fig

Figure(layout=FigureLayout(height='420px'))

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [52]:
ideal_cities = city_df.loc[(city_df['Max Temp'] < 75) & (city_df['Max Temp'] > 40) & (city_df['Wind Speed'] < 20) & (city_df['Cloudiness'] < 20)]

ideal_cities = ideal_cities.dropna(axis=0, how='any')
ideal_cities = ideal_cities.reset_index(drop=True)
ideal_cities.head()

,City,Cloudiness,Country,Date,Humidity,Lat,Lng,Max Temp,Wind Speed
0,zhuhai,0,CN,1558378755,39,40.71,112.04,44.60,4.47
1,flinders,0,AU,1558378755,62,-34.58,150.85,60.01,11.41
2,pushkinskiye gory,6,RU,1558378756,70,57.02,28.91,65.04,6.93
3,castro,0,CL,1558378756,70,-42.48,-73.76,48.20,9.17
4,kodinsk,14,RU,1558378758,81,58.60,99.18,48.84,4.97


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [53]:
hotel_df = ideal_cities.loc[:, ['City', 'Country', 'Lat', 'Lng', 'Cloudiness', 'Max Temp', 'Wind Speed', 'Humidity']]
hotel_df['Hotel Name'] = ""
hotel_df.head()

,City,Country,Lat,Lng,Cloudiness,Max Temp,Wind Speed,Humidity,Hotel Name
0,zhuhai,CN,40.71,112.04,0,44.60,4.47,39,
1,flinders,AU,-34.58,150.85,0,60.01,11.41,62,
2,pushkinskiye gory,RU,57.02,28.91,6,65.04,6.93,70,
3,castro,CL,-42.48,-73.76,0,48.20,9.17,70,
4,kodinsk,RU,58.60,99.18,14,48.84,4.97,81,


In [54]:
url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"
params = {'type': 'hotel',
         'keyword': 'hotel',
         'radius': 5000,
         'key': g_key}
for index, row in hotel_df.iterrows():
    name = row['City']
    lat = row['Lat']
    lng = row['Lng']
    params['location'] = f"{lat},{lng}"
    res = requests.get(url, params=params).json()
    print(f"Retrieving Hotel Data for: {name}.")
    result = res['results']
    try:
        hotel_df.loc[index, 'Hotel Name'] = result[0]['name']
        print(f"The nearest hotel to {name} is: {result[0]['name']}.")
    except:
        print(f"Could not find a hotel in {name}... skipping to next city.")
    
        

Retrieving Hotel Data for: zhuhai.
Could not find a hotel in zhuhai... skipping to next city.
Retrieving Hotel Data for: flinders.
The nearest hotel to flinders is: Lakeview Hotel Motel.
Retrieving Hotel Data for: pushkinskiye gory.
The nearest hotel to pushkinskiye gory is: Arina R..
Retrieving Hotel Data for: castro.
The nearest hotel to castro is: Enjoy Chiloé. Hotel De La Isla.
Retrieving Hotel Data for: kodinsk.
The nearest hotel to kodinsk is: Art-Otel' " Kupecheskiy".
Retrieving Hotel Data for: ambon.
The nearest hotel to ambon is: Hotel de la Plage - Damgan - Morbihan - Bretagne.
Retrieving Hotel Data for: qingdao.
The nearest hotel to qingdao is: Crowne Plaza Qingdao.
Retrieving Hotel Data for: varhaug.
The nearest hotel to varhaug is: Huset ved havet.
Retrieving Hotel Data for: kodiak.
Could not find a hotel in kodiak... skipping to next city.
Retrieving Hotel Data for: mar del plata.
The nearest hotel to mar del plata is: Patagonia Hotel.
Retrieving Hotel Data for: kolyvan.


Retrieving Hotel Data for: hakkari.
The nearest hotel to hakkari is: Vaihmalan Hovi.
Retrieving Hotel Data for: la seyne-sur-mer.
The nearest hotel to la seyne-sur-mer is: ibis budget Toulon Centre.
Retrieving Hotel Data for: veraval.
The nearest hotel to veraval is: B&B HOTEL.
Retrieving Hotel Data for: tongren.
The nearest hotel to tongren is: Tongren Grand Metro Park Hotel.


In [55]:
hotel_df

,City,Country,Lat,Lng,Cloudiness,Max Temp,Wind Speed,Humidity,Hotel Name
0,zhuhai,CN,40.71,112.04,0,44.60,4.47,39,
1,flinders,AU,-34.58,150.85,0,60.01,11.41,62,Lakeview Hotel Motel
2,pushkinskiye gory,RU,57.02,28.91,6,65.04,6.93,70,Arina R.
3,castro,CL,-42.48,-73.76,0,48.20,9.17,70,Enjoy Chiloé. Hotel De La Isla
4,kodinsk,RU,58.60,99.18,14,48.84,4.97,81,"Art-Otel' "" Kupecheskiy"""
...,...,...,...,...,...,...,...,...,...
84,viedma,AR,-40.81,-62.99,14,58.56,10.98,41,Hotel Austral Viedma
85,hakkari,FI,61.34,23.75,0,69.80,5.82,46,Vaihmalan Hovi
86,la seyne-sur-mer,FR,43.10,5.88,0,69.01,2.24,72,ibis budget Toulon Centre
87,veraval,FR,49.65,0.71,0,61.00,8.05,62,B&B HOTEL


In [56]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [57]:
# Add marker layer ontop of heat map
marker_layer = gmaps.marker_layer(locations, info_box_content = hotel_info)
fig.add_layer(marker_layer)
fig

# Display figure


Figure(layout=FigureLayout(height='420px'))